In [1]:
%gui asyncio
import asyncio
import ipywidgets as widgets
from threading import Timer
from pandas import DataFrame
import qgrid
from datetime import timedelta, datetime

In [2]:
names=['scrambled', 'sunny side up', 'poached', 'over easy']
ini_val = 'sunny side up'
list(zip(names,[0]*len(names)))

[('scrambled', 0), ('sunny side up', 0), ('poached', 0), ('over easy', 0)]

In [3]:
df = DataFrame( data=zip(names,[0]*len(names)), columns=['name','time']) #, dtype=int 90,60,30
df.set_index('name', inplace=True)
qw = qgrid.show_grid( df, show_toolbar=False)

In [4]:
slider = widgets.SelectionSlider(
    options=names,
    value=ini_val,
    description='I like my eggs ...',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    style = {'description_width': 'initial'},
    layout = widgets.Layout( width='100%')
)

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result( (change.old, change.new) )
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

s = datetime.now()
df['s'] = timedelta(seconds=0)

async def f():
    global s, df
    for i in range(10):
        print('did work %s'%i)
        #df = qw.get_changed_df()
        old, new = await wait_for_change(slider, 'value')
        now = datetime.now()
        d = now - s
        df.loc[old,'s'] += d
        df.loc[old,'time'] += round(d.total_seconds(),1)
        s = now
        #df.loc[old,'s'].round('s')
        df.s = df.s.round('s')
        #print('d',type(d),'df',type(df.loc[old,'s']))
        qw.df = df
        qw.change_selection([new])
        print('async function continued with value %s,%s'%(old, new))

asyncio.ensure_future(f())
widgets.HBox([slider,qw])

did work 0


In [5]:
qw.change_selection([ini_val])

did work 0


In [10]:
df = qw.get_changed_df()
df.s = df.s.round('s')
display(df)

,time,s
name,,
scrambled,86.3,0 days 00:01:26
sunny side up,14.7,0 days 00:00:15
poached,7.0,0 days 00:00:07
over easy,139.4,0 days 00:02:19
